In [1]:
from pyhanlp import *

In [2]:
a = '你好，欢迎在Python中调用HanLP的API，美国国会要杀了他 你信吗？'

In [3]:
for i in HanLP.segment(a):
    print(i)

你好/vl
，/w
欢迎/v
在/p
Python/nx
中/f
调用/v
HanLP/nx
的/ude1
API/nx
，/w
美国/nsf
国会/nis
要/v
杀/v
了/ule
他/rr
 /w
你/rr
信/n
吗/y
？/w


In [4]:
Config = JClass("com.hankcs.hanlp.HanLP$Config")

In [5]:
Config.ShowTermNature = False

In [6]:
CRFSegment = JClass("com.hankcs.hanlp.seg.CRF.CRFSegment")

In [7]:
segment = CRFSegment().enableCustomDictionary(True)

java.lang.IllegalArgumentExceptionPyRaisable: java.lang.IllegalArgumentException: CRF分词模型加载 /home/xy/anaconda3/lib/python3.6/site-packages/pyhanlp/static/data/model/segment/CRFSegmentModel.txt 失败，耗时 4 ms